<a href="https://colab.research.google.com/github/trishitadhara/NLP/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 16.2MB/s 
     |████████████████████████████████| 3.0MB 19.3MB/s 
     |████████████████████████████████| 890kB 30.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=baf9da70042f1cb31994907d820ce2a0d79b1f10ca9266219a766e500b898133
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
test1feeds = pd.read_csv("drive/My Drive/Test-1_feeds.csv")

In [ ]:
for i in range(14499):
  if(test1feeds['tweet'][i][1]=="'"):
    test1feeds['tweet'][i]=test1feeds['tweet'][i].replace("'","\"")
  test1feeds['tweet'][i]=test1feeds['tweet'][i][2:-2]
  test1feeds['tweet'][i]=test1feeds['tweet'][i].split('", "')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [ ]:
print(test1feeds.head)

<bound method NDFrame.head of           id                                              tweet
0       3849  [👨🏻👨🏻 https://t.co/I9KVh2IN1s, RT @TMobile: .@...
1        957  [Not too much drama, but it’s still DRAMA 😏 Ki...
2      14388  [I’m happy to announce I’ll be watching your a...
3       7446  [Back in the 6 last night. Thank you Toronto. ...
4       1107  [So excited for the announcement tomorrow from...
...      ...                                                ...
14494  33530  [Loving being home after an amazing trip away ...
14495  29315  [Last training session in Rio for the aussie_r...
14496  36954  [Welcome to Sri Lanka @PMOIndia, Congratulatio...
14497   4554  [RT @jkenney: Sad to learn that Flora MacDonal...
14498   4512  [Wisconsin the choice is clear. Who represents...

[14499 rows x 2 columns]>


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

In [ ]:
#regular expressions
smile_re = re.compile("(:\)|;\)|:-\)|;-\)|:\(|:-\(|:-o|:o|<3)")
emoji_re = re.compile("(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff])")
not_ascii_re = re.compile("([^\x00-\x7F]+)")
url_re = re.compile("https?[\S]*")
mention_re = re.compile("@[\w]*[\W\s]")
hashtag_re = re.compile("#[\w]*[\W]")
numbers_re = re.compile("(^|\D)[\d]+[.'\d]*\D")
id_re = re.compile("\"id\":\s[\d]*}")
symbols_re = re.compile("[!?:*+\[\]\\\/.'\":;,\_()%$#@!]")
dash_re = re.compile("[\s]+-[\s]+")
extraspace_re = re.compile("\s+")


In [ ]:
MAX_TWEET_PER_USER=500
def feeds_preprocess(profile):
        count=0
        processed_tweets=[]
        for tweet in profile:
            tweet = tweet.replace("\\n","")
            tweet = re.sub(smile_re," [EMOTICON] ",tweet)
            tweet = re.sub(emoji_re," [EMOJI] ",tweet)
            tweet = re.sub(not_ascii_re," ",tweet)
            tweet = re.sub(url_re," [URL] ",tweet)
            tweet = re.sub(mention_re," [USER] ",tweet)
            tweet = re.sub(hashtag_re," [HASHTAG] ",tweet)
            tweet = re.sub(numbers_re," [NUMBERS] ",tweet)   #keep this if u want to tag numericals as <NUMBERS>
            tweet = re.sub(id_re,"",tweet)
            tweet = re.sub(symbols_re," ",tweet)
            tweet = re.sub(dash_re,"",tweet)
            #tweet = re.sub(retweet_re," ",tweet)
            processed_tweets.append(tweet)
            if count<MAX_TWEET_PER_USER:
                count+=1
            else:
                break        
        concat_tweets =""
        for tweet in processed_tweets:
            concat_tweets = concat_tweets +" [SEP] " + tweet
        concat_tweets = re.sub(extraspace_re," ",concat_tweets)
        return concat_tweets

In [ ]:
test1feeds["processed_tweet"]=test1feeds["tweet"].apply(feeds_preprocess)

In [ ]:
print(test1feeds["processed_tweet"].head)

<bound method NDFrame.head of 0         [SEP] URL [SEP] RT USER USER and Jimmy are up...
1         [SEP] Not too much drama but it EMOJI s still...
2         [SEP] I EMOJI m happy to announce I EMOJI ll ...
3         [SEP] Back in the NUMBERS last night Thank yo...
4         [SEP] So excited for the announcement tomorro...
                               ...                        
14494     [SEP] Loving being home after an amazing trip...
14495     [SEP] Last training session in Rio for the au...
14496     [SEP] Welcome to Sri Lanka PMOIndia [SEP] Con...
14497     [SEP] RT USER Sad to learn that Flora MacDona...
14498     [SEP] Wisconsin the choice is clear Who repre...
Name: processed_tweet, Length: 14499, dtype: object>


In [ ]:
from keras.preprocessing.sequence import pad_sequences
def encoded_sent(sent):
  #print(len(sent.split()),sent)
  s=tokenizer.encode(sent,add_special_tokens=False)
  padded_sent=[]
  #print(s)
  if len(s)<511:
    padded_sent = pad_sequences([s[1:]], maxlen=510, dtype="long", value=0, truncating="post", padding="post")
    padded_sent = list(padded_sent[0])
  else:
    padded_sent+=s[1:129]
    padded_sent+=s[-382:]
    
  padded_sent=[101]+padded_sent
  padded_sent=padded_sent+[102]
  #print(len(padded_sent),padded_sent)
  return padded_sent

Using TensorFlow backend.


In [ ]:
print(test1feeds['processed_tweet'][0])
print(encoded_sent(test1feeds['processed_tweet'][0]))
print(encoded_sent("hello hi this is me"))

Token indices sequence length is longer than the specified maximum sequence length for this model (2685 > 512). Running this sequence through the model will result in indexing errors


 [SEP] URL [SEP] RT USER USER and Jimmy are up to something The surprises continue tomorrow on HASHTAG at NUMBERS 30 EST on NBC HASHTAG EMOJI [SEP] The end is FIRES What EMOJI s not to love URL [SEP] Meet the newest Bieber my baby sister Bay Bieber URL [SEP] Make sure you guys go see HASHTAG this weekend So good Congrats jonmchu [SEP] URL [SEP] Just Skate URL [SEP] NO BRAINER out now URL URL [SEP] RT USER NUMBERS 9 NUMBERS HASHTAG NUMBERS 9 NUMBERS his Friday [SEP] URL [SEP] URL [SEP] Thank you USER [SEP] EMOTICON URL [SEP] HASHTAG URL [SEP] RT USER Justin Bieber Cast As Voice Talent In Animated Film About Cupid From Mythos Studios URL URL EMOJI [SEP] We will never let go of hope HASHTAG URL [SEP] RT USER Never forgotten Forever honored URL [SEP] Praying for Santa Fe [SEP] RT USER USER has surpassed NUMBERS BILLION views across all official channels on YouTube He is the most viewed artist in HISTOR EMOJI [SEP] New game don EMOJI t fall URL URL [SEP] Proud of my brother USER on his new 

In [ ]:
test1feeds["tokenized_tweet"]=test1feeds["processed_tweet"].apply(encoded_sent)

Streaming output truncated to the last 5000 lines.
Token indices sequence length is longer than the specified maximum sequence length for this model (5495 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5127 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (4114 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (5740 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is lo

In [ ]:
test1feeds.to_csv("drive/My Drive/encoded_test1_feeds.csv")

In [ ]:

print(test1feeds['tokenized_tweet'][0])

[101, 24471, 2140, 102, 19387, 5310, 5310, 1998, 5261, 2024, 2039, 2000, 2242, 1996, 20096, 3613, 4826, 2006, 23325, 15900, 2012, 3616, 2382, 9765, 2006, 6788, 23325, 15900, 7861, 29147, 2072, 102, 1996, 2203, 2003, 8769, 2054, 7861, 29147, 2072, 1055, 2025, 2000, 2293, 24471, 2140, 102, 3113, 1996, 14751, 12170, 22669, 2026, 3336, 2905, 3016, 12170, 22669, 24471, 2140, 102, 2191, 2469, 2017, 4364, 2175, 2156, 23325, 15900, 2023, 5353, 2061, 2204, 26478, 8609, 2015, 6285, 12458, 6979, 102, 24471, 2140, 102, 2074, 17260, 24471, 2140, 102, 2053, 4167, 2121, 2041, 2085, 24471, 2140, 24471, 2140, 102, 19387, 5310, 3616, 1023, 3616, 23325, 15900, 3616, 1023, 3616, 2010, 5958, 102, 24471, 2140, 102, 24471, 2140, 102, 4067, 2017, 5310, 102, 7861, 20214, 2239, 24471, 2140, 102, 23325, 15900, 7861, 29147, 2072, 102, 2651, 24471, 2140, 102, 2017, 2074, 2288, 6573, 1996, 6616, 2041, 102, 19387, 5310, 1037, 2261, 2062, 2420, 2000, 2131, 2115, 9735, 2000, 1996, 20930, 12679, 5178, 2283, 2007, 5310,

In [ ]:
def attention(tokens):
  att_mask=[]
  for token in tokens:
    #print(token)
    if token > 0:
      att_mask.append(1)
    else:
      att_mask.append(0)
  #print(att_mask)
  return att_mask

In [ ]:
attention(test1feeds['tokenized_tweet'][100])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
test1feeds["attention"]=test1feeds["tokenized_tweet"].apply(attention)

In [ ]:
test1feeds.to_csv("drive/My Drive/encoded2_test1_feeds.csv")

NameError: ignored

In [ ]:
print(test1feeds["attention"].head)

<bound method NDFrame.head of 0        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
                               ...                        
14494    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
14495    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
14496    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
14497    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
14498    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
Name: attention, Length: 14499, dtype: object>


In [ ]:
trainfeeds = pd.read_csv("/content/drive/My Drive/encoded2_test1_feeds.csv")

In [ ]:
print(trainfeeds['tokenized_tweet'].head)

<bound method NDFrame.head of 0        [101, 24471, 2140, 102, 19387, 5310, 5310, 199...
1        [101, 2025, 2205, 2172, 3689, 2021, 2009, 7861...
2        [101, 1045, 7861, 29147, 2072, 1049, 3407, 200...
3        [101, 2067, 1999, 1996, 3616, 2197, 2305, 4067...
4        [101, 2061, 7568, 2005, 1996, 8874, 4826, 2013...
                               ...                        
14494    [101, 8295, 2108, 2188, 2044, 2019, 6429, 4440...
14495    [101, 2197, 2731, 5219, 1999, 5673, 2005, 1996...
14496    [101, 6160, 2000, 5185, 7252, 7610, 28765, 903...
14497    [101, 19387, 5310, 6517, 2000, 4553, 2008, 100...
14498    [101, 5273, 1996, 3601, 2003, 3154, 2040, 5836...
Name: tokenized_tweet, Length: 14499, dtype: object>


In [ ]:
def f(x):
  a=x.replace("[","")
  a=a.replace("]","")
  arr=a.split(", ")
  ar=[]
  for i in arr:
    i=int(i)
    ar.append(i)
  return ar

In [ ]:
y=f(trainfeeds['tokenized_tweet'][0])
print(y)
print(y[2])
trainfeeds["tokenized_tweet_int"]=trainfeeds["tokenized_tweet"].apply(f)

[101, 24471, 2140, 102, 19387, 5310, 5310, 1998, 5261, 2024, 2039, 2000, 2242, 1996, 20096, 3613, 4826, 2006, 23325, 15900, 2012, 3616, 2382, 9765, 2006, 6788, 23325, 15900, 7861, 29147, 2072, 102, 1996, 2203, 2003, 8769, 2054, 7861, 29147, 2072, 1055, 2025, 2000, 2293, 24471, 2140, 102, 3113, 1996, 14751, 12170, 22669, 2026, 3336, 2905, 3016, 12170, 22669, 24471, 2140, 102, 2191, 2469, 2017, 4364, 2175, 2156, 23325, 15900, 2023, 5353, 2061, 2204, 26478, 8609, 2015, 6285, 12458, 6979, 102, 24471, 2140, 102, 2074, 17260, 24471, 2140, 102, 2053, 4167, 2121, 2041, 2085, 24471, 2140, 24471, 2140, 102, 19387, 5310, 3616, 1023, 3616, 23325, 15900, 3616, 1023, 3616, 2010, 5958, 102, 24471, 2140, 102, 24471, 2140, 102, 4067, 2017, 5310, 102, 7861, 20214, 2239, 24471, 2140, 102, 23325, 15900, 7861, 29147, 2072, 102, 2651, 24471, 2140, 102, 2017, 2074, 2288, 6573, 1996, 6616, 2041, 102, 19387, 5310, 1037, 2261, 2062, 2420, 2000, 2131, 2115, 9735, 2000, 1996, 20930, 12679, 5178, 2283, 2007, 5310,

In [ ]:
trainfeeds["attention_int"]=trainfeeds['attention'].apply(f)

In [ ]:
trainlabels=pd.read_csv("/content/drive/My Drive/Test-1_labels.csv")

In [ ]:
train_y=trainlabels[['by','fame','gen','occ']]


In [ ]:
train_x=trainfeeds[['tokenized_tweet_int','attention_int']]

In [ ]:
from sklearn import preprocessing
le_occ = preprocessing.LabelEncoder()
le_gen = preprocessing.LabelEncoder()
le_fame = preprocessing.LabelEncoder()
le_by = preprocessing.LabelEncoder()


In [ ]:
trainlabels['occ_tokenized']=le_occ.fit_transform(trainlabels['occ'])
trainlabels['gen_tokenized']=le_gen.fit_transform(trainlabels['occ'])


In [ ]:
trainlabels['fame_tokenized']=le_fame.fit_transform(trainlabels['occ'])
trainlabels['by_tokenized']=le_by.fit_transform(trainlabels['occ'])

In [ ]:
gen = torch.tensor(trainlabels['gen_tokenized'].values)

In [ ]:
def f2(x):
  return x+1
trainlabels['occ_tokenized']=trainlabels['occ_tokenized'].apply(f2)

In [ ]:
print(train_x.head)

<bound method NDFrame.head of                                      tokenized_tweet_int                                      attention_int
0      [101, 24471, 2140, 102, 19387, 5310, 5310, 199...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1      [101, 2025, 2205, 2172, 3689, 2021, 2009, 7861...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2      [101, 1045, 7861, 29147, 2072, 1049, 3407, 200...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
3      [101, 2067, 1999, 1996, 3616, 2197, 2305, 4067...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
4      [101, 2061, 7568, 2005, 1996, 8874, 4826, 2013...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
...                                                  ...                                                ...
14494  [101, 8295, 2108, 2188, 2044, 2019, 6429, 4440...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
14495  [101, 2197, 2731, 5219, 1999, 5673, 2005, 1996...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..

In [ ]:
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_x,gen, 
                                                            random_state=2018, test_size=0.1)

In [ ]:
print(train_labels)

tensor([2, 7, 6,  ..., 1, 2, 7])


In [ ]:
train_id=train_inputs['tokenized_tweet_int']
train_mask=train_inputs['attention_int']
val_id=validation_inputs['tokenized_tweet_int']
val_mask=validation_inputs['attention_int']

In [ ]:
print(train_id)

967      [101, 1045, 2572, 5627, 2000, 2507, 2039, 1037...
8279     [101, 5310, 1045, 2052, 2707, 2007, 1037, 2003...
5095     [101, 12699, 2398, 2007, 7815, 2003, 2028, 199...
7770     [101, 19387, 5310, 2079, 2017, 2066, 12041, 78...
2952     [101, 7052, 5310, 16122, 4596, 2007, 2026, 244...
                               ...                        
7324     [101, 2017, 2210, 5053, 24471, 2140, 102, 1938...
917      [101, 2272, 6170, 23067, 3070, 2000, 3976, 323...
2825     [101, 6429, 2008, 5310, 2081, 2019, 2886, 4748...
10854    [101, 3773, 10958, 3616, 7249, 1045, 2175, 201...
9466     [101, 19387, 5310, 1037, 7438, 2005, 2035, 237...
Name: tokenized_tweet_int, Length: 13049, dtype: object


In [ ]:
train_id=train_id.apply(torchconvert)

NameError: ignored

In [ ]:
train_id=train_id.values
#print(train_id)
train_id=list(train_id)
train_id=torch.tensor(train_id)
print(train_id)

tensor([[  101,  1045,  2572,  ..., 24471,  2140,   102],
        [  101,  5310,  1045,  ..., 24471,  2140,   102],
        [  101, 12699,  2398,  ..., 24471,  2140,   102],
        ...,
        [  101,  6429,  2008,  ..., 24471,  2140,   102],
        [  101,  3773, 10958,  ..., 13102,  8485,   102],
        [  101, 19387,  5310,  ..., 15150,  2015,   102]])


In [ ]:

train_mask=torch.tensor(list(train_mask.values))
val_mask=torch.tensor(list(val_mask.values))
val_id=torch.tensor(list(val_id.values))


In [ ]:
print(train_mask)

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])


In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 8, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 8

# Create the DataLoader for our training set.
train_data = TensorDataset(train_id, train_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(val_id, val_mask, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  1,632.    Elapsed: 0:00:19.
  Batch    80  of  1,632.    Elapsed: 0:00:37.
  Batch   120  of  1,632.    Elapsed: 0:00:55.
  Batch   160  of  1,632.    Elapsed: 0:01:13.
  Batch   200  of  1,632.    Elapsed: 0:01:31.
  Batch   240  of  1,632.    Elapsed: 0:01:49.
  Batch   280  of  1,632.    Elapsed: 0:02:08.
  Batch   320  of  1,632.    Elapsed: 0:02:26.
  Batch   360  of  1,632.    Elapsed: 0:02:44.
  Batch   400  of  1,632.    Elapsed: 0:03:02.
  Batch   440  of  1,632.    Elapsed: 0:03:20.
  Batch   480  of  1,632.    Elapsed: 0:03:38.
  Batch   520  of  1,632.    Elapsed: 0:03:57.
  Batch   560  of  1,632.    Elapsed: 0:04:15.
  Batch   600  of  1,632.    Elapsed: 0:04:33.
  Batch   640  of  1,632.    Elapsed: 0:04:51.
  Batch   680  of  1,632.    Elapsed: 0:05:09.
  Batch   720  of  1,632.    Elapsed: 0:05:28.
  Batch   760  of  1,632.    Elapsed: 0:05:46.
  Batch   800  of  1,632.    Elapsed: 0:06:04.
  Batch   840  of